# MIMIX demo

In [1]:
include("models.jl")  # ignore method definition warnings from module Graphs

samplers (generic function with 2 methods)

## Load the data

In [2]:
Y = readcsv("data/Y.csv", Int)  # sequence counts

166×2662 Array{Int64,2}:
  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  3  0  0  0  0  0  0   0  0
  0  0  1  0  0  0  0  0  0  0  0  0     0  0  0  1  0  0  0  0  0  0   0  0
  2  0  0  0  0  0  0  0  0  0  0  0     0  0  0  1  0  0  0  0  0  0   0  0
  1  0  2  0  0  0  0  0  0  0  0  0     0  0  0  2  0  0  0  0  0  0   0  0
  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0   0  0
 26  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  1  0  0  0  0  0  0   0  0
  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0   0  0
  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  2  0  0  0  0  0  0  18  0
 30  0  4  0  0  0  0  0  0  0  0  0     0  0  0  3  1  0  0  0  0  0   0  0
  3  0  0  0  0  0  0  0  0  0  0  0     0  0  0  1  0  0  0  0  0  0   0  0
  0  0  3  0  0  0  0  0  0  0  0  0  …  0  0  0  2  0  0  0  0  0  0   0  0
  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0   0  0
  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  2

In [3]:
X = readcsv("data/X.csv", Int)  # experimental treatments (fencing, supplement, both)

166×3 Array{Int64,2}:
 0  0  0
 0  0  0
 0  0  0
 0  0  0
 0  1  0
 0  1  0
 0  1  0
 0  1  0
 1  0  0
 1  0  0
 1  0  0
 1  0  0
 1  1  1
 ⋮      
 0  0  0
 0  0  0
 0  1  0
 0  1  0
 0  1  0
 1  0  0
 1  0  0
 1  0  0
 1  1  1
 1  1  1
 1  1  1
 1  1  1

In [4]:
Z = readcsv("data/Z.csv", Int)  # site and block-within-site effects

166×2 Array{Int64,2}:
 1   5
 1   5
 1   5
 1   5
 1   5
 1   5
 1   5
 1   5
 1   5
 1   5
 1   5
 1   5
 1   5
 ⋮    
 4  15
 4  15
 4  15
 4  15
 4  15
 4  15
 4  15
 4  15
 4  15
 4  15
 4  15
 4  15

##  Reduce the dimensionality
For the purposes of this demo, we reduce the dimensionality of $Y$ to achieve a quicker runtime. We remove any taxon that does not constitute >10% of at least one sample in the dataset.

In [5]:
proportions = Y ./ sum(Y, 2)
Y = Y[:, vec(any(proportions .> 1e-1, 1))]
Y

166×39 Array{Int64,2}:
 0    7    2    18    9     8  0  …     7  0     5     75  0  0    16  3
 0    5   92  3584    9    17  0       69  0     6    439  1  0   280  1
 0   10    1    85    8   215  0      116  0     3     57  0  0  1055  1
 0   38    2  4523   23     6  0        8  0    13    501  0  0    20  2
 0    6    8   169    4     2  0       52  1     3    118  2  0    10  0
 0    5    2    77    9     4  0  …     7  0     4    499  0  0  1026  1
 0    9    0    32    8     8  0       16  1     6    116  0  0    26  0
 0    9    8    39    7    17  0       20  0     6    405  0  0  3632  2
 0   23  297   530   26    91  0     1175  1    32   3114  0  0    13  3
 0   14    3   612   18    15  0        8  0     6    985  1  0    15  1
 0  120   20  2394    7    31  0  …     5  0     8   1294  2  0   211  2
 0   12    1   307   14     1  0       40  1     1    420  3  0    13  0
 0   77    0    19   13     9  0      136  1    14  12620  2  0   569  2
 ⋮                          

## MCMC settings

In [6]:
settings = Dict(
    # posterior sampling
    :iters => 30000,
    :burnin => 15000,
    :thin => 1,
    
    # parameters to save: 
    # β (treatment effects on individual taxa)
    :monitor => [:β], 
    
    # Hamiltonian monte carlo settings
    :hmc_verbose => false,  # if true, print accept rate to console
    :epsilon => 0.05,       # step size of HMC, tune for accept b/w 25%-45%
    :steps => 16,           # number of leapfrog steps of HMC, tune for accept b/w 25-45%
);

## Fit MIMIX

In [7]:
srand(881)
sim = fit(MIMIX(30), Y, X, Z; settings...)

MCMC Simulation of 30000 Iterations x 1 Chain...

Chain 1:   0% [4:26:44 of 4:26:49 remaining]
Chain 1:  10% [0:30:50 of 0:34:15 remaining]
Chain 1:  20% [0:26:01 of 0:32:31 remaining]
Chain 1:  30% [0:22:22 of 0:31:57 remaining]
Chain 1:  40% [0:19:00 of 0:31:41 remaining]
Chain 1:  50% [0:15:45 of 0:31:31 remaining]
Chain 1:  60% [0:12:36 of 0:31:31 remaining]
Chain 1:  70% [0:09:33 of 0:31:50 remaining]
Chain 1:  80% [0:06:25 of 0:32:04 remaining]
Chain 1:  90% [0:03:14 of 0:32:24 remaining]
Chain 1: 100% [0:00:00 of 0:32:41 remaining]



Object of type "Mamba.ModelChains"

Iterations = 15001:30000
Thinning interval = 1
Chains = 1
Samples per chain = 15000

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0250799 -1.31736e-5; 0.0 0.0 … 3.42906e-5 1.81504e-5]

## Identify significant effects on taxa

In [14]:
q = quantile(sim)

              2.5%            25.0%              50.0%              75.0%                 97.5%        
g_var[1] 12.4190011722 17.9140354960473829 21.7592319959504 26.789506360442196353 36.220628855148774505
g_var[2]  2.3571494966  3.5834250584508442  4.2814498426445  4.919687926991024973  6.136728685676414941
  β[1,1] -0.0853414300  0.0000000000000000  0.0000000000000  0.000000000000000000  0.077130188644120135
  β[2,1] -0.0242579044  0.0000000000000000  0.0000000000000  0.000000000000000000  0.072894527986600816
  β[3,1] -0.0284720592  0.0000000000000000  0.0000000000000  0.000000000000000000  0.059345952936055782
  β[4,1] -0.0219752803  0.0000000000000000  0.0000000000000  0.000000000000000000  0.049352130879757164
  β[5,1] -0.0836281592  0.0000000000000000  0.0000000000000  0.000000000000000000  0.037898919924762270
  β[6,1] -0.0527023759  0.0000000000000000  0.0000000000000  0.000000000000000000  0.029333620261886635
  β[7,1] -0.0395502795  0.0000000000000000  0.0000000000000  0.0

In [15]:
lb = q.value[:, 1]
ub = q.value[:, 5]
is_signif = !(lb .< 0 .< ub)
sim.names[is_signif]

7-element Array{AbstractString,1}:
 "g_var[1]"
 "g_var[2]"
 "β[2,2]"  
 "β[5,2]"  
 "β[20,2]" 
 "β[31,2]" 
 "β[32,2]" 

Taxa 2, 5, 20, 31, and 32 are significantly affected by the second treatment of the experiment, nutrient supplement to the soil.